Install required libraries

In [ ]:
!pip install wandb
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.1)


Please make sure that you mount your Google Drive so that you can access the datasets of images in the EECS 504 Shared Folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [ ]:
import os
import sys
import torch
import torchvision
from torchvision import transforms, datasets
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
import wandb #Import this to collab
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import torchvision.utils as vutils
import torch.nn as nn
import torch.optim as optim
#from config import config
import matplotlib.animation as animation
from IPython.display import HTML
from sklearn.metrics import accuracy_score

Hyperparameter

In [ ]:
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64
# ngf=256

# Size of feature maps in discriminator
ndf = 64
# ndf=256

# Number of training epochs
num_epochs = 10

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Save per * steps
save_interval = 10

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Number of layers in the loop in discriminator
dn = 3
# Number of layers in the loop in generator
gn = 3

# Digit we are training
digit = 1

In [ ]:
hparams = {'networks': {'discriminator': {'in_dim': nc, 'h_dims': ndf, 'num_layers': dn},
                       'generator': {'in_dim': nz, 'h_dims': ngf, 'out_dim': nc, 'num_layers': gn}},
          'device': 'gpu',
          'exp': 'experiment_name',
          'batch_size': batch_size,
          'optim': 'Adam', 
          'lr': lr,
          'epochs': num_epochs,
          'save_interval': save_interval}

Connect to GPU

In [ ]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")

Using the GPU!


# Load data

Load SVHN Dataset - run this dataset

In [ ]:
# Import from our Shared Google Drive
# Set directories for training and testing datasets
root_dir = "/content/drive/Shareddrives/EECS 504 Shared Drive/Datasets/SVHN Dataset/"
train_dir = root_dir + "train/" + str(digit) + "/" 
test_dir = root_dir + "test/" + str(digit) + "/"

# Images are 32 x 32
img_size = 32*(dn-)
image_size = 32

# Create transforms
train_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Create datasets
train_img = datasets.ImageFolder(train_dir, transform=train_transform)
test_img = datasets.ImageFolder(test_dir, transform=test_transform)

Load Freiburg Groceries dataset

Transformation


In [ ]:
# Import from our Shared Google Drive
# Set directories for training and testing datasets
# root_dir = "/content/drive/Shareddrives/EECS 504 Shared Drive/Datasets/Freiburg Groceries Dataset/"
# train_dir = root_dir + "train/"
# test_dir = root_dir + "test/"

# # Images are 256 x 256
# img_size = 256
# image_size = 256

# # Create transforms
# train_transform = transforms.Compose([
#     transforms.Resize(img_size),
#     transforms.CenterCrop(img_size),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])
# test_transform = transforms.Compose([
#     transforms.Resize(img_size),
#     transforms.CenterCrop(img_size),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

# # Create datasets
# train_img = datasets.ImageFolder(train_dir, transform=train_transform)
# test_img = datasets.ImageFolder(test_dir, transform=test_transform)

Create the loaders

In [ ]:
# may need to change batch sizes
trainloaders = torch.utils.data.DataLoader(train_img, batch_size=batch_size, shuffle=True)
testloaders = torch.utils.data.DataLoader(test_img, batch_size=batch_size, shuffle=True)

Explore the dataset (optional)

In [ ]:
print("train_img type   :",type(train_img))
print("train_img length :",len(train_img))
# print("test_img length :",len(test_img))
print("train_img classes:",train_img.classes)
print("train_img[0] type:",type(train_img[0]))
print("train_img[0][0] t:",type(train_img[0][0]))
print("train_img[0][1] t:",type(train_img[0][1]))
print("train_img[0][0] s:",train_img[0][0].size())
print("train_img[0][1]  :",train_img[0][1])

n_labels = len(train_img.classes)

train_img type   : <class 'torchvision.datasets.folder.ImageFolder'>
train_img length : 4220
train_img classes: ['1']
train_img[0] type: <class 'tuple'>
train_img[0][0] t: <class 'torch.Tensor'>
train_img[0][1] t: <class 'int'>
train_img[0][0] s: torch.Size([3, 32, 32])
train_img[0][1]  : 0


Display images

In [ ]:
def imshowIndividual(data_image, tensor=False):
    image = data_image[0]
    label = data_image[1]
    image = image.numpy().transpose((1, 2, 0))

    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5],)
    image = std * image + mean
    image = np.clip(image, 0, 1)

    plt.imshow(image)
    plt.show()
    
    print(train_img.classes[label])

In [ ]:
def imshowIndividualGAN(data_image, tensor=False):
    image = data_image

    image = image.numpy().transpose((1, 2, 0))

    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5],)
    image = std * image + mean
    image = np.clip(image, 0, 1)

    plt.imshow(image)
    plt.show()

In [ ]:
def plotGridImages(loaders, batchSize, title):
  # Plot grid
  real_batch = next(iter(loaders))
  plt.figure(figsize=(8, 16))
  plt.axis("off")
  plt.title(title)
  plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:batch_size], padding=2, normalize=True).cpu(),(1,2,0)))
  print(real_batch[1])

# Build Discriminator and Generator
The generator is shown in the image below:


In [ ]:
#Build disciminator and generator
manualSeed=999
torch.manual_seed(manualSeed)


class Discriminator(nn.Module):
    """
    Discriminator. Linear layers, hidden layer activation is RELU, Output activation is Sigmoid
    in_dim: a scalar, dimension of input
    h_dims: a list of dimensions of hidden layers

    ngpu: Number of GPUs available. Use 0 for CPU mode.
    """

    """
    General layout:
    self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    """

    # def __init__(self, in_dim, h_dims):
    def __init__(self, in_dim, h_dims, num_layers, ngpu = 1): # Added on Apr.12th.2021

        nc = in_dim # Number of channels in the training images, integer. For color images this is 3
        ndf = h_dims # Size of feature maps in discriminator

        #Trying Sequential
        super(Discriminator, self).__init__()
        #self.ngpu = ngpu
        lst=[]

        # First layer
        lst.append(nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        lst.append(nn.LeakyReLU(0.2, inplace=True))

        # num_layers hidden layers
        for i in range(0,num_layers): # 0 to 2
            current_input = ndf * 2**i
            current_output = ndf * 2**(i+1)
            lst.append(nn.Conv2d(current_input, current_output, 4, 2, 1, bias=False))
            lst.append(nn.BatchNorm2d(current_output))
            lst.append(nn.LeakyReLU(0.2, inplace=True))

        # Last layer        
        lst.append(nn.Conv2d(current_output, 1, 2, 1, 0, bias=False)) # changed kernel from 4 to 2
        lst.append(nn.Sigmoid())

        self.main = nn.Sequential(*lst)

        # define out_activation
        self.out_activation = nn.Sigmoid()

    def forward(self, x):
        return self.main(x)


class Generator(nn.Module):
    """
    Generator. Linear layer, hidden layer activation is RELU, Output activation is Tanh
    in_dim: a scalar, dimension of input
    h_dims: a list of dimensions of hidden layers
    out_dim: a scalar, dimensin of output

    ngpu: Number of GPUs available. Use 0 for CPU mode.
    """

    """
    General layout:
    self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )
    """

    def __init__(self, in_dim, h_dims, out_dim, num_layers, ngpu = 1):
        
        super(Generator, self).__init__()
        
        number_of_hidden_layers = gn # 3 hidden layers
        nz = in_dim # Size of z latent vector (i.e. size of generator input)
        ngf = h_dims # Size of feature maps in generator
        nc = out_dim

        # Initial layer
        # lst=[torch.nn.ConvTranspose2d(nz, ngf * 2**number_of_hidden_layers, 4, 1, 0, bias=False)]
        # For 32 x 32 image
        lst=[torch.nn.ConvTranspose2d(nz, ngf * 2**number_of_hidden_layers, 2, 1, 0, bias=False)] # changed kernel from 4 to 2
        lst.append(nn.BatchNorm2d(ngf * 2**number_of_hidden_layers))
        lst.append(nn.ReLU(inplace=True))
       
        # num_layers hidden layers
        for i in range(num_layers,0,-1):
            lst.append(nn.ConvTranspose2d(ngf*2**i, ngf*2**(i-1), 4, 2, 1, bias=False)) #check h_dim length
            lst.append(nn.BatchNorm2d(ngf*2**(i-1)))
            lst.append(nn.ReLU(inplace=True))

        # Final layer
        lst.append(nn.ConvTranspose2d(ngf, nc, 4,2,1))
        lst.append(nn.Tanh())

        self.main = nn.Sequential(*lst)    

    def forward(self, x):
        return self.main(x)

In [ ]:
def weights_init(m):
  
  if type(m) == nn.ConvTranspose2d:
    nn.init.normal_(m.weight.data,0.0,0.02)
  elif type(m) == nn.Conv2d:
    nn.init.normal_(m.weight.data,0.0,0.02)
  elif type(m) == nn.BatchNorm2d:
    nn.init.normal_(m.weight.data, 1.0, 0.02)
    nn.init.constant_(m.bias.data, 0)
  #  classname = m.__class__.__name__
  #  if classname.find('Conv') != -1:
  #       nn.init.normal_(m.weight.data, 0.0, 0.02)
  #  elif classname.find('BatchNorm') != -1:
  #       nn.init.normal_(m.weight.data, 1.0, 0.02)
  #       nn.init.constant_(m.bias.data, 0)


In [ ]:
## Code for initializing conv, relu and batch norm layers with 0 mean and 0.02 stddev
netG = Generator(nz, ndf, nc,gn).to(device)
netG.apply(weights_init)
netD = Discriminator(nc, ndf, dn).to(device)
netD.apply(weights_init)
print(netG)
print(netD)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): Tanh()
  )
)
Discriminato

Print out parameters

In [ ]:
num_params = sum([item.numel() for item in netG.parameters() if item.requires_grad])
print(num_params)
num_params = sum([item.numel() for item in netD.parameters() if item.requires_grad])
print(num_params)

In [ ]:
#Build the GAN

# class GAN:
#     """
#     Generative Adversarial Network. Encapsulates a generator network, a discriminator network,
#     functionality for generating with the generator, and losses for training both networks.
#     """
#     def __init__(self, hparams):

#         #Discriminator from given hyperparameters
#         self.discriminator = Discriminator(in_dim=hparams['networks']['discriminator']['in_dim'], # 3
#                                                     #h_dims=hparams['networks']['discriminator']['h_dims'], # 64
#                                                     #num_layers=hparams['networks']['discriminator']['num_layers']) # 3
        
#         #Generator from given hyperparameters
#         self.generator = Generator(in_dim=hparams['networks']['generator']['in_dim'], # 100
#                                             #h_dims=hparams['networks']['generator']['h_dims'], # 64
#                                             #out_dim=hparams['networks']['generator']['out_dim'], # 3
#                                             #num_layers=hparams['networks']['generator']['num_layers']) # 3
        
#         # Apply weights to discriminator and generator
#         self.discriminator.apply(weights_init)
#         self.generator.apply(weights_init)

        

#         #Move to gpu if available
#         if hparams['device'] == 'gpu':
#             self.generator.cuda()
#             self.discriminator.cuda()

#         self.hparams = hparams

#     def generate(self, num_samples):
#         z = torch.randn((num_samples, self.hparams['networks']['generator']['in_dim'], 1, 1)) # Second argument is nz, or size of generator input (100)
#         if self.hparams['device'] == 'gpu':
#             z = z.cuda()
#         return self.generator(z)

    #Loss for discriminator
    #def loss_d(self, x, x_g):
        
        # real_logits = self.discriminator(x)
        # # print('real logits shape: ', real_logits.shape)
        # fake_logits = self.discriminator(x_g)
        # # print('fake logits shape: ', fake_logits.shape)
        # # print('real logits values: ', real_logits)
        # # print('fake logits values: ', fake_logits)
        # logits = torch.vstack((real_logits, fake_logits)) # get to 256 x 1
        # logits = torch.squeeze(logits)
        # # print('new logits shape: ', logits.shape)
        # targets = torch.vstack((torch.zeros((real_logits.shape[0], 1)),
        #                         torch.ones((fake_logits.shape[0], 1))))
        # targets = torch.squeeze(targets)
        # # print("targets: ", targets)
        # if self.hparams['device'] == 'gpu':
        #     targets = targets.cuda()
        # # accuracy = accuracy_score(self.discriminator.out_activation(targets).detach().cpu().numpy().round(),
        # #                           self.discriminator.out_activation(logits).detach().cpu().numpy().round())
        # # discriminator out activation is Sigmoid
        # # print('targets shape: ', targets.shape)
        # # print('logits shape: ', logits.shape)
        # y_true = self.discriminator.out_activation(targets).detach().cpu().numpy().round()
        
        # # print('y true shape: ',y_true.shape)
        # y_pred = self.discriminator.out_activation(logits).detach().cpu().numpy().round()
        # y_pred = logits
        # # print('y pred shape: ',y_pred.shape)
        # # y_true = real_logits
        # # y_pred = fake_logits
        # # y_pred = y_pred[:,:]
        # # print('y pred shape: ',y_pred.shape)
        # accuracy = accuracy_score(y_true, y_pred)
        # return torch.nn.BCELoss()(logits, targets), accuracy
    
    #Loss for generator
    #def loss_g(self, x_g):
        # fake_logits = self.discriminator(x_g)
        # fake_logits = torch.squeeze(fake_logits)
        # targets = torch.zeros((fake_logits.shape[0], 1))
        # targets = torch.squeeze(targets)
        # # print('fake_logits shape: ', fake_logits.shape)
        # # print('targets shape: ', targets.shape)
        # if self.hparams['device'] == 'gpu':
        #     targets = targets.cuda()
        # return torch.nn.BCELoss()(fake_logits, targets)
  

In [ ]:
img_list = [] # for saving images
fixed_noise = torch.randn(128, nz, 1, 1).cuda()
criterion = nn.BCELoss()

In [ ]:

# def train(hparams):
#     # Setup experiment.
#     # wandb.init(project='EECS 504 - Final', name=hparams['exp'], config=hparams)

#     # Load model.
#     model = GAN(hparams)
#     # wandb.watch(model.generator)
#     # wandb.watch(model.discriminator)

#     # Load data.
#     # data = load_mnist_data(batch_size=hparams['batch_size'] // 2)
#     # we use trainloaders as the training data
#     data = trainloaders

#     # Optimizers.
#     optimizer = getattr(torch.optim, hparams['optim'])
#     optimizer_d = optimizer(params=model.discriminator.parameters(), lr=hparams['lr'], betas=(beta1, 0.999))
#     optimizer_g = optimizer(params=model.generator.parameters(), lr=hparams['lr'], betas=(beta1, 0.999))

#     # Train.
#     for epoch in range(hparams['epochs']):
#         for batch_idx, (x, _) in enumerate(data):
#             # Flatten and migrate to GPU
#             # x = torch.flatten(x, start_dim=1)
#             if hparams['device'] == 'gpu':
#                 x = x.cuda()

#             # Train discriminator.
#             netG.zero_grad()
#             x_g = netG(num_samples=x.shape[0]).detach()
#             # print(x.shape)
#             # print(x_g.shape)
#             # output = model.discriminator(x)
#             # fake_output = model.discriminator(x_g)
#             output_real=netD(x).view(-1)
#             output_fake=netD(x_g).view(-1)        
#             label_real=0.
#             label_real=torch.full((x.shape[0],), 1., dtype=torch.float).cuda()
#             label_fake=torch.full((x_g.shape[0],), 0., dtype=torch.float).cuda()

            
#             loss_d_real=criterion(output_real,label_real) 
           
#             loss_d_real.backward()
#             loss_d_fake=criterion(output_fake,label_fake) 
#             loss_d_fake.backward()
#             loss_d =loss_d_real + loss_d_fake

#             optimizer_d.step()
            

#             # Train with fake 

#             # Train generator.
#             netD.zero_grad()
#             output_fake=netD(x_g).view(-1)
#             label=torch.full((x_g.shape[0],), 1., dtype=torch.float).cuda()   
#             loss_g = criterion(output_fake,label)
           
#             loss_g.backward()
#             optimizer_g.step()

#             # Logging.
#             print("epoch: {}/{} | batch: {}/{} | loss_g: {} | loss_d: {} %".format(
#                 epoch, hparams['epochs'], batch_idx, len(data), loss_g, loss_d )) #, acc_d * 100
#             # wandb.log({
#             #     "loss_g": loss_g,
#             #     "loss_d": loss_d,
#             #     "acc_d": acc_d,
#             # })

#             # Save generated images
#             with torch.no_grad():
#               fake = model.generator(fixed_noise).detach().cpu()
#               print(fake.shape)
#               if batch_idx % 10 == 0:
#                 imshowIndividualGAN(fake[1,:,:,:])
#             img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

#         # Checkpointing and saving samples.
#             # if epoch % hparams['save_interval'] == 0:
#             # if batch_idx == 10:
#             #     with torch.no_grad():
#             #         examples = model.generate(num_samples=64)
#             #     examples = (examples.reshape(-1, 28, 28) + 1) * 255 / 2
#             #     examples = np.round(examples.detach().cpu().numpy())[:64]
#             #     fig = plt.figure(figsize=(64, 64))
#             #     grid = ImageGrid(fig, 111, nrows_ncols=(8, 8), axes_pad=1)
#             #     for i, (ax, im) in enumerate(zip(grid, examples)):
#             #         ax.imshow(np.reshape(im, (28, 28)), cmap=plt.get_cmap('gray'))
#             #     break
#             #     epoch = 6
#                 # wandb.log({'Generated samples': plt})
#     torch.save(model.generator.state_dict(),'/content/saved_generator_'  + str(sub) +'.pt' ) 


In [ ]:
# def main():
#     # train(config) #Need to define config 
#     train(hparams)


# if __name__ == '__main__':
#     main()

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
real_label = 1.
fake_label = 0.
# model = GAN(hparams)
netG = Generator(nz, ndf, nc,gn).to(device)
netG.apply(weights_init)
netD = Discriminator(nc, ndf, dn).to(device)
netD.apply(weights_init)
optimizer = getattr(torch.optim, hparams['optim'])
optimizer_d = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))

#optimizer(params=model.discriminator.parameters(), lr=hparams['lr'], betas=(beta1, 0.999))
optimizer_g = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))#optimizer(params=model.generator.parameters(), lr=hparams['lr'], betas=(beta1, 0.999))

dataloader = trainloaders
# model = GAN(hparams)

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device) # x
        b_size = real_cpu.size(0) # x.shape[0]
        label = torch.full((b_size,), real_label, dtype=torch.float).cuda()
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1) # output_real
        # Calculate loss on all-real batch
        errD_real = criterion(output, label) # loss_d_real
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1).cuda()
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizer_d.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label) # real labels
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizer_g.step()
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1

# save last image
torch.save(img_list[-1], str(digit)+ '.pt')
# save generator
torch.save(netG.state_dict(),'saved_generator_'  + str(digit) +'.pt' ) 

Starting Training Loop...
[0/10][0/33]	Loss_D: 1.5655	Loss_G: 2.8254	D(x): 0.3955	D(G(z)): 0.4295 / 0.0664


KeyboardInterrupt: ignored

In [ ]:
## This part of the code will save 128 Image grid into 
## individual images under the 'Fake Images' folder in shared drive
## NOTE: Make suree to change the folder digit name before running
for i in range(len(generated_image)):
  single_image = generated_image[i,:,:,:]
  torchvision.utils.save_image(single_image,"/content/drive/Shareddrives/EECS 504 Shared Drive/Fake Images/5/"+str(i)+".jpg")

In [ ]:
print(len(img_list))
print(img_list[11].shape)
imshowIndividualGAN(img_list[99])

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

Function to show images - called "plotGridImages" defined above


# Classification (K Nearest Neighbors)





Be sure to run everything before and including "Loading Data" part

Image processing

In [ ]:
def knn_image_proc(train_img):
  '''
  Image flattening and normalization for kNN
  Input:
    train_img: images and labels
  Output:
    x_train: processed images (flattened to be vectors and normalized)
    y: labels
  '''
  n_train = len(train_img)

  x_train_raw = []

  y = []

  for i in range(n_train):
    # x_i = train_img[i][0].reshape(-1)
    x_i = np.array(train_img[i][0].reshape(-1))
    x_train_raw.append(x_i)
    y.append(train_img[i][1])

  # Normalize to zero mean so covariance is just matrix multiplication
  mean = np.mean(x_train_raw, axis=0, keepdims=True)
  x_train = x_train_raw - mean 

  return x_train, y 

Reduction

In [ ]:
def compute_basis(data, n=300):
  """ 
  Computing basis for image reduction
  Inputs:
      data: multi-dimensional array of size number of images * number of pixels. Each row is a flatten image.
      n: number of modes to be kept
  Output:
      eigenvectors: basis
  """
  
  eigenvectors = None

  sig = np.matmul(np.transpose(data),data)
  _, eigenvectors = np.linalg.eig(sig)
  eigenvectors = eigenvectors[:, range(n)]

  return eigenvectors

kNN

In [ ]:
# # Self defined kNN function
# def knn_predict(eig_train_x, eig_test_x, y_train, y_test, k=1):

#   """Implement the KNN algorithm. The output should be a vector containing your predictions"""

#   predictions = np.zeros_like(y_test)
    
#   N, D = eig_train_x.shape
#   for i in range(len(y_test)):
#     d = np.ones((N,))
#     for j in range(N):
#       dx = eig_train_x[j,:] - eig_test_x[i,:]
#       d[j] = np.linalg.norm(dx)

#     x_in = np.argsort(d)
#     neb_in = x_in[:k]
#     neb_y = y_train[neb_in]

#     counts = np.bincount(neb_y)
#     predictions[i] = np.argmax(counts)

#   return predictions

Implementations

In [ ]:
k = 5 # k nearest neighbors
if_POD = True # Do we do POD?

# Image flattening and normalization
x_train, y = knn_image_proc(train_img)

# Image reduction
if if_POD:
  eigenvectors = compute_basis(x_train, n=300)
  x_input = np.matmul(x_train,eigenvectors)
else:
  x_input = x_train

neigh = KNeighborsClassifier(n_neighbors = k)

neigh.fit(x_input, y)

In [ ]:
neigh = neighbors.KNeighborsClassifier(n_neighbors = k)

Visualization

In [ ]:
i_plot = 0
imshowIndividual(train_img[0])
print()